In [1]:
!pip install git+https://github.com/nasa/pretrained-microscopy-models

  Cloning https://github.com/nasa/pretrained-microscopy-models to /tmp/pip-req-build-ko1vjpyf
  Running command git clone --filter=blob:none --quiet https://github.com/nasa/pretrained-microscopy-models /tmp/pip-req-build-ko1vjpyf
  Resolved https://github.com/nasa/pretrained-microscopy-models to commit 1a4896ca66a61bd50c2e660087cce18bfa9ce896
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 13.5 MB/s eta 0:00:00
  Created wheel for pretrained_microscopy_models: filename=pretrained_microscopy_models-0.1.0-py3-none-any.whl size=14911 sha256=b877a81920b689909e0e27e8c4098bc776e5f133876afa4402572b32af327bbf
  Stored in directory: /tmp/pip-ephem-wheel-cache-fodv3_65/wheels/f9/40/40/ce55fa400a3c27b11c43cd9e

In [2]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import torch
import gc
from torchvision import transforms
from PIL import Image
import os
import pandas as pd
import math

from numpy import mean
from numpy import std
from numpy import absolute

from sklearn.model_selection import train_test_split, LeaveOneOut, cross_val_score, KFold
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import make_scorer,r2_score
from sklearn.svm import SVR

from transformers import CLIPProcessor, CLIPModel, AutoModel, CLIPImageProcessor, SamModel, SamProcessor, AutoProcessor

#import torch
import pretrained_microscopy_models as pmm
import torch.utils.model_zoo as model_zoo
import torch.nn as nn



/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=False)
url = pmm.util.get_pretrained_microscopynet_url('resnet50', 'micronet')


state_dict = torch.hub.load_state_dict_from_url(url, map_location=torch.device('cpu'))
device = 'cuda' if torch.cuda.is_available() else 'cpu'



Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://nasa-public-data.s3.amazonaws.com/microscopy_segmentation_models/resnet50_pretrained_microscopynet_v1.1.pth.tar" to /root/.cache/torch/hub/checkpoints/resnet50_pretrained_microscopynet_v1.1.pth.tar
100%|██████████| 97.8M/97.8M [00:06<00:00, 14.9MB/s]


In [5]:
transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                 ])

In [8]:
def make_NASA_features(img):
    # print (img.shape)
    # plt.imshow(img)
    # plt.show()

    x_shape = img.shape[0]
    y_shape = img.shape[1]

    if x_shape <= y_shape:
      min = x_shape
    else:
      min = y_shape

    img_new = img[:min, :min, :]

    img_rgb = Image.fromarray(img_new).convert('RGB')

    #print (img_rgb.size)

    img_t = transform(img_rgb)

    # image = img_t.permute(1, 2, 0)
    # plt.imshow(image)
    # plt.show()

    img_t = img_t.unsqueeze(0)

    # Extract everything up to the avgpool layer
    feature_extractor = nn.Sequential(*list(model.children())[:-1])
    feature_extractor.eval()


    with torch.no_grad():
         features = feature_extractor(img_t)
    features = np.array(features)
    features = np.squeeze(features)

    #print (features.shape)
    return features

In [9]:
image_path = '/content/drive/MyDrive/11_12/ImagesToKeep'

desired_files = [file for file in os.listdir(image_path) if not file.startswith('.')]

count  = 0
for filename in desired_files:
    f = filename[:-4]

    img = Image.open(image_path + '/' + filename)

    img = np.array(img)

    img_features = make_NASA_features(img)

    new_filename =  '/content/drive/MyDrive/11_12/ImagesToKeep_Features/' + f + '_Nasa.npy'
    np.save(new_filename, img_features)
    print (count)

    count = count + 1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
